In [5]:
! pip install ratelimit

import json
import pandas as pd
import numpy as np
from ratelimit import limits, sleep_and_retry
from urllib.request import urlopen

!pip install pytrends
!pip install fredapi
!pip install quandl
import matplotlib.pyplot as plt
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.stats import linregress
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import pickle
from pandas.tseries.offsets import BMonthEnd
import json
import lightgbm as lgb
import datetime
from datetime import timedelta
from sklearn.metrics import multilabel_confusion_matrix
from scipy.stats import norm, kurtosis, skew
import math
import warnings
warnings.simplefilter("ignore", UserWarning)
pd.options.mode.chained_assignment = None  # default='warn'
import scipy.optimize as sco
from functools import reduce
import time
import gc
from pytrends.request import TrendReq
from fredapi import Fred
import quandl
import gc
import matplotlib.pyplot as plt

api = '86c3322708a8cb34c8f59b85318f8da8'
quandl.ApiConfig.api_key = 'ezG_brK9DueLj9Xsdv1r'
fred = Fred(api_key=api)

  Created wheel for ratelimit: filename=ratelimit-2.2.1-cp37-none-any.whl size=5908 sha256=d898c52bc6517ed408a17d640a6a2ee84bf61f4a4f9aa4c3bb3b35e57bbf2dd1
  Stored in directory: /root/.cache/pip/wheels/05/d9/82/3c6044cf1a54aab9151612458446d9b17a38416869e1b1d9b8
Successfully built ratelimit


In [ ]:
DIRECTORY = '/content/drive/My Drive/Trading/' # сюда написать рабочую директорию куда сохранять все файлы
START_DOWNLOAD = '1990-01-01'
END_DOWNLOAD = '2021-06-01'
API_FINANCIALMODELINGPREP = '37d9e9f7c27eb5b3bf7a63e606b030ed'

### Скачиваем котировки, pl, bs, key metrics, financial growth по всем тикерам, торгующимся на NASDAQ

In [ ]:
ONE_MINUTE = 60 

@sleep_and_retry
@limits(calls=300, period=ONE_MINUTE)
def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

def reduce_mem_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)


# print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(data.memory_usage(index=True).sum())))

In [ ]:
# загружаем список всех доступных тикеров

url = "https://financialmodelingprep.com/api/v3/stock/list?apikey={}".format(API_FINANCIALMODELINGPREP)
symbols = pd.DataFrame(get_jsonparsed_data(url))

# отбираем тикеры для скачки только те, что торгуются на насдаке
stock_exchange_list = [ 'Nasdaq Global Select',
                       'Nasdaq Global Market',
                       'Nasdaq Capital Market', 'NASDAQ', 'Nasdaq', 
                       'NasdaqGS',
                       'NasdaqGM'
                        ]

symbols_to_download = symbols[symbols['exchange'].isin(stock_exchange_list)]['symbol'].sort_values().values   
len(symbols_to_download)

In [ ]:
# скачиваем и схохраняем исторические цены по выбранным тикерам за указанный период

url = "https://financialmodelingprep.com/api/v3/historical-price-full/{}?from={}&to=&apikey={}".format(START_DOWNLOAD, END_DOWNLOAD, API_FINANCIALMODELINGPREP) 

historical_price_full = []
skipped_symbols = []
for symbol in symbols_to_download:
  try:
    tmp = pd.json_normalize(get_jsonparsed_data(url.format(symbol))['historical'])
  except:
    skipped_symbols.append(symbol)
    pass
    
  if not tmp.empty:
    tmp['symbol'] = symbol
    historical_price_full.append(reduce_mem_usage(tmp))

df_full = pd.concat(historical_price_full)

with open('{}historical_price_last_year.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(df_full, f) 

In [ ]:
# Определяем ссылки для прочей информации

links = {}

links['pl'] = "https://financialmodelingprep.com/api/v3/income-statement/{}?from={}&to={}&apikey={}" # PL
links['bs'] = "https://financialmodelingprep.com/api/v3/balance-sheet-statement/{}?from={}}&to={}}&apikey={}" # BS
links['financial_growth'] = "https://financialmodelingprep.com/api/v3/financial-growth/{}?from={}}&to={}}&apikey={}" # financial_growth
links['key_metrics'] = "https://financialmodelingprep.com/api/v3/key-metrics/{}?from={}&to={}&apikey={}" # key_metrics


# скачиваем всю прочую информацию

df_dict = {}

for link in ['pl', 'bs', 'financial_growth', 'key_metrics']:
  df = []
  for symbol in symbols_to_download:
    url = links[link].format(symbol, START_DOWNLOAD, END_DOWNLOAD, API_FINANCIALMODELINGPREP)
    df.append(pd.DataFrame(get_jsonparsed_data(url)))

  with open('{}{}_last_year.pickle'.format(DIRECTORY,link), 'wb') as f:
    pickle.dump(pd.concat(df), f) 

  print(link, 'OK')

  df_dict[link] = df

### Обогащаем датасет с помощью feature - engineering 

In [2]:
def merge_with_calendar(quotes):
  # Поскольку могут быть торговые дни без сделок, данных по таким дням нет, однако, объем сделок = 0 нужен, чтобы корректно считать скользящую по объему!
  # В качестве календаря для мерджа берется список дней в которых торговался эпл (предполагаем что в каждом торговом дне у эпла были сделки)
  dates = pd.DataFrame(quotes[quotes['symbol']=='AAPL']['date'])
  quotes_df = pd.DataFrame([])

  for ticker in quotes['symbol'].unique():

    if quotes_df.empty:

      quotes_df = dates.merge(quotes[quotes['symbol']==ticker], how='left', on='date')
      quotes_df['symbol'] = quotes_df['symbol'].fillna(method='bfill')
      quotes_df.dropna(axis=0, subset=['symbol'], inplace=True)
    
    else:

      tmp = dates.merge(quotes[quotes['symbol']==ticker], how='left', on='date')
      tmp['symbol'] = tmp['symbol'].fillna(method='bfill')
      tmp.dropna(axis=0, subset=['symbol'], inplace=True)
      quotes_df = quotes_df.append(tmp)

  return quotes_df.drop_duplicates()

def fill_na_days(quotes_df):
  # Поскольку могут быть торговые дни без сделок, данных по таким дням нет, однако, объем сделок = 0 нужен, чтобы корректно считать скользящую по объему!
  quotes_df[['volume', 'unadjustedVolume', 'change', 'changePercent', 'changeOverTime']] = quotes_df[['volume', 'unadjustedVolume', 'change', 'changePercent', 'changeOverTime']].fillna(0)
  quotes_df[['open', 'high', 'low', 'close', 'adjClose', 'vwap']] = quotes_df[['open', 'high', 'low', 'close', 'adjClose', 'vwap']].fillna(method='bfill')

  return quotes_df

def total_return_from_returns(returns):
  # считает месячное изменение на основе дневных изменений в %
  return (returns + 1).prod() - 1

def target_category(pct_change):
  # делаем таргет
  if pct_change >= 0.1:
    return 0
  if pct_change >= 0.05 and pct_change < 0.1:
    return 1
  if pct_change >= 0 and pct_change < 0.05: 
    return 2
  if pct_change >= -0.1 and pct_change < 0:
    return 3
  if pct_change < -0.1:
    return 4

def calc_pct_change(data):
  return (data.iloc[-1] - data.iloc[0]) / data.iloc[0]

def next_month_return_and_target_alt(quotes_df):

  quotes_df['date'] = pd.to_datetime(quotes_df['date'])
  quotes_df['year'] = quotes_df['date'].dt.year
  quotes_df['month'] = quotes_df['date'].dt.month

  # считаем изменение за текущий календарный месяц и делаем сдвиг назад на 1 месяц
  res = quotes_df[['symbol', 'year', 'month', 'close']].groupby(['symbol', 'year', 'month']).apply(calc_pct_change).reset_index()
  res['close'] = res[['symbol', 'close']].groupby('symbol').shift(-1)
  res.columns = ['symbol', 'year', 'month', 'price_change_next_calendar_month']

  # мерджим результат с основным дф
  quotes_df = quotes_df.merge(res, how='left', on=['symbol', 'year', 'month'])

  # считаем категорию для таргета
  quotes_df['target'] = quotes_df['price_change_next_calendar_month'].apply(target_category)

  return quotes_df


def next_month_return_and_target(quotes_df, col='daily_pct_change'):
  # считаем изменение за следующий торговый месяц в %
  quotes_df['date'] = pd.to_datetime(quotes_df['date'])
  quotes_df['year'] = quotes_df['date'].dt.year
  quotes_df['month'] = quotes_df['date'].dt.month

  # считаем изменение в % за текущий торговый месяц
  month_changes = quotes_df.reset_index().set_index("date")
  month_changes = month_changes.groupby(['symbol', 'year', 'month'])[[col]].apply(total_return_from_returns)
  month_changes = month_changes.reset_index()

  # Изменение цены в % за следующий торговый месяц
  month_changes['price_change_next_calendar_month'] = month_changes.groupby('symbol')[col].shift(-1)

  # считаем категорию для таргета
  month_changes['target'] = month_changes['price_change_next_calendar_month'].apply(target_category)

  quotes_df = quotes_df.merge(month_changes[['symbol', 'year', 'month', 'price_change_next_calendar_month', 'target']], on=['symbol', 'year', 'month'], how='left')

  return quotes_df.dropna(axis=0, subset=['target'])

def business_days(quotes_df):
  offset = BMonthEnd()
  #Last day of current month
  quotes_df['last_bday_current_month'] = quotes_df['date'].apply(lambda x: offset.rollforward(x))
  quotes_df['is_last_bday_current_month'] = (quotes_df['last_bday_current_month'] == quotes_df['date'])
  # Last day of previous month
  # quotes_df['last_bday_previous_month'] = quotes_df['date'].apply(lambda x: offset.rollback(x))

  return quotes_df

def daily_returns(quotes_df):
  return quotes.groupby('symbol')['close'].pct_change()

def wma_calc(data, period):
  return data.ewm(span=period, adjust=False).mean()

def VOSC(quotes_df, short_period=12, long_period=26):
  VEMA12 = quotes_df[['symbol', 'volume']].groupby('symbol').apply(lambda x: wma_calc(x, short_period)).reset_index(drop=True) #
  VEMA26 = quotes_df[['symbol', 'volume']].groupby('symbol').apply(lambda x: wma_calc(x, long_period)).reset_index(drop=True) #
  VOSC = (VEMA26['volume'] - VEMA12['volume']) / VEMA12['volume'] * 100
  return VOSC

def VMACD2(quotes_df, short_period=12, long_period=26, signal_period=9):
  quotes_df['VEMA12'] = quotes_df[['symbol', 'volume']].groupby('symbol').apply(lambda x: wma_calc(x, short_period)).reset_index(drop=True)['volume']
  quotes_df['VEMA26'] = quotes_df[['symbol', 'volume']].groupby('symbol').apply(lambda x: wma_calc(x, long_period)).reset_index(drop=True)['volume']
  quotes_df['MACD'] = (quotes_df['VEMA12'] - quotes_df['VEMA26']) / quotes_df['volume'] * 100
  quotes_df['signal'] = quotes_df[['symbol', 'MACD']].groupby('symbol').apply(lambda x: wma_calc(x, signal_period)).reset_index(drop=True)['MACD']
  quotes_df['signal'] = quotes_df['signal'] / quotes_df['volume'] * 100
  quotes_df['MACD_diff'] = quotes_df['MACD'] - quotes_df['signal']
  return quotes_df['MACD_diff']

def calc_pct_change(data):
  return (data.iloc[-1] - data.iloc[0]) / data.iloc[0]

def ROC(quotes_df, period=20, column='close'):
  return quotes_df[['symbol', column]].groupby('symbol').rolling(period, min_periods=2).apply(calc_pct_change).reset_index()[column]

def VOLUME(quotes_df, period=20):
  # Объем торгов за день / Объем торгов за последние 20 дней MEAN * Скорость возврата за последние 20 дней MEAN
  mean_volume_20 = quotes_df[['symbol', 'volume']].groupby('symbol').rolling(period, min_periods=2).mean().reset_index()['volume']

  return quotes_df['volume'] / mean_volume_20 * quotes_df['ROC']

def Price1M(quotes_df):

  mean_close_21 = quotes_df[['symbol', 'close']].groupby('symbol').rolling(21, min_periods=2).mean().reset_index()['close']

  return quotes_df['close'] / mean_close_21 - 1

def MAC(quotes_df, period=20):

  MA20 = quotes_df[['symbol', 'close']].groupby('symbol').rolling(period, min_periods=2).mean().reset_index()['close']

  return MA20 / quotes_df['close']

def VARIANCE(quotes_df, period=20, column='daily_pct_change'):
  return quotes_df[['symbol', column]].groupby('symbol').rolling(period, min_periods=2).apply(lambda x: np.var(x)).reset_index()[column]

def KURTOSIS(quotes_df, period=20):
  return quotes_df[['symbol', 'close']].groupby('symbol').rolling(period, min_periods=2).apply(lambda x: kurtosis(x)).reset_index()['close']

def SKEW(quotes_df, period=20):
  return quotes_df[['symbol', 'close']].groupby('symbol').rolling(period, min_periods=2).apply(lambda x: skew(x)).reset_index()['close']

def TRIX(quotes_df, period=10):
  quotes_df['EMA1'] = quotes_df[['symbol', 'close']].groupby('symbol').apply(lambda x: wma_calc(x, period=10)).reset_index(drop=True)['close']
  quotes_df['EMA2'] = quotes_df[['symbol', 'EMA1']].groupby('symbol').apply(lambda x: wma_calc(x, period=10)).reset_index(drop=True)['EMA1']
  quotes_df['EMA3'] = quotes_df[['symbol', 'EMA2']].groupby('symbol').apply(lambda x: wma_calc(x, period=10)).reset_index(drop=True)['EMA2']
  TRIX10 = quotes_df[['symbol', 'EMA3']].groupby('symbol').pct_change()
  return TRIX10

def money_ratio(data):
  positive_money_flow = sum(data[data>0])
  negative_money_flow = sum(data[data<0])

  if negative_money_flow == 0:
    negative_money_flow = 1

  money_ratio = - positive_money_flow / negative_money_flow

  return 100 - (100 / (1 + money_ratio))

def MFI(quotes_df, period=14):
  typical = (quotes_df['high'] + quotes_df['low'] + quotes_df['close']) / 3
  money_flow = pd.DataFrame(typical * quotes_df['volume'])
  money_flow['symbol'] = quotes_df['symbol']
  money_flow['pct_change'] = money_flow.groupby('symbol').pct_change().fillna(0)
  money_flow.loc[money_flow['pct_change'] >= 0, 'sign'] = 1
  money_flow.loc[money_flow['pct_change'] < 0, 'sign'] = -1
  money_flow['money_flow'] = money_flow[0] * money_flow['sign']
  return money_flow[['symbol', 'money_flow']].groupby('symbol').rolling(period, min_periods=2).apply(money_ratio).reset_index()['money_flow']

def boll_up(quotes_df, D=2, period=20):
  SMA = quotes_df[['symbol', 'close']].groupby('symbol').rolling(period, min_periods=2).mean().reset_index()[['symbol', 'close']]
  STD = quotes_df[['symbol', 'close']].groupby('symbol').rolling(period, min_periods=2).std().reset_index()[['symbol', 'close']]
  return (SMA['close'] + D*STD['close']) / quotes_df['close']

def boll_down(quotes_df, D=2, period=20):
  SMA = quotes_df[['symbol', 'close']].groupby('symbol').rolling(period, min_periods=2).mean().reset_index()[['symbol', 'close']]
  STD = quotes_df[['symbol', 'close']].groupby('symbol').rolling(period, min_periods=2).std().reset_index()[['symbol', 'close']]
  return (SMA['close'] - D*STD['close']) / quotes_df['close']

def CAGR(data):
  data = list(data)
  days_to_year = len(data)/251
  return (data[-1]/data[0])**(1/days_to_year) - 1

def SHARPE_RATIO(quotes_df, period=20, risk_free_rate=0.04):
  VOL = quotes_df[['symbol', 'close']].groupby('symbol').rolling(period, min_periods=period).std().reset_index()[['symbol', 'close']]
  RP = quotes_df[['symbol', 'close']].groupby('symbol').rolling(period, min_periods=period).apply(CAGR).reset_index()[['symbol', 'close']] # считаем среднее дневное изменение за период
  RP['close'] = RP['close'] - risk_free_rate # числитель коэффициента шарпа
  return RP['close'] / VOL['close'] * math.sqrt(252) # 252 торговых дня в году = годовая доходность

def load_fundamentals(quotes_df):
  with open('{}financial_growth_last_year.pickle'.format(DIRECTORY), 'rb') as f: #fin_growth
    fin_growth = pickle.load(f)
  with open('{}key_metrics_last_year.pickle'.format(DIRECTORY), 'rb') as f: #key_metrics
    key_metrics = pickle.load(f)
  with open('{}pl_last_year.pickle'.format(DIRECTORY), 'rb') as f: #financial-statement_annual
    PL = pickle.load(f)
  with open('{}bs_last_year.pickle'.format(DIRECTORY), 'rb') as f: #balance-sheet_annual
    balance_sheet_annual = pickle.load(f)


  fundamentals_data = PL[['date', 'symbol', 'period','fillingDate', 'acceptedDate',
                          'ebitdaratio', 'operatingIncomeRatio', 
                          'incomeBeforeTaxRatio', 'netIncomeRatio', 'eps', 
                          'weightedAverageShsOut',
                          'ebitda', 'incomeBeforeTax', 'interestExpense', 'revenue']]
                          
  fundamentals_data = fundamentals_data.merge(fin_growth, 
                                              on=['date', 'symbol', 'period'], 
                                              how='left') 
   
  fundamentals_data = fundamentals_data.merge(key_metrics, 
                                              on=['date', 'symbol', 'period'], 
                                              how='left')
  
  fundamentals_data = fundamentals_data.merge(balance_sheet_annual[['symbol','fillingDate',
                                                                    'totalStockholdersEquity', 
                                                                    'retainedEarnings', 
                                                                    'longTermDebt']], 
                                              on=['fillingDate', 'symbol'], 
                                              how='left')

  fundamentals_data.rename(columns={'date': 'period_end'}, inplace=True)
  fundamentals_data['date'] = pd.to_datetime(fundamentals_data['acceptedDate'])

  return quotes_df.merge(fundamentals_data, how='left', 
                         on=['date', 'symbol'])

def load_profile(quotes_df):
  with open('{}profile_full.pickle'.format(DIRECTORY), 'rb') as f: 
    profile = pickle.load(f)

  profile['exchangeShortName'] = profile['exchangeShortName'].astype('category')
  profile['industry'] = profile['industry'].astype('category')
  profile['ceo'] = profile['ceo'].astype('category')
  profile['sector'] = profile['sector'].astype('category')
  profile['country'] = profile['country'].astype('category')
  profile['fullTimeEmployees'] = profile['fullTimeEmployees'].fillna(0).replace({'': 0}).astype(int)
  
  return quotes_df.merge(profile[['symbol', 'exchangeShortName', 'industry',
                           'ceo', 'sector', 'country', 'fullTimeEmployees',
                           'isActivelyTrading', 'isEtf']], on='symbol', how='left')

def p_b_ratio_daily(quotes_df):
  book = quotes_df['totalStockholdersEquity'] / quotes_df['weightedAverageShsOut']
  price = quotes_df['close']
  return book / price

def p_b_ratio_daily_alter(quotes_df):
  return quotes_df['bookValuePerShare'] / quotes_df['close']

def net_profit_to_total_operate_revenue_ttm(quotes_df):
  return quotes_df['netIncomePerShare'] / quotes_df['revenuePerShare']

def market_cap_daily(quotes_df):
  return quotes_df['weightedAverageShsOut'] * quotes_df['close']

def VOL(quotes_df, period=20):
  # Среднее значение 20-дневной оборачиваемости, в% (объем сделок поделить на всего капитализацию)
  quotes_df['vol_daily'] = quotes_df['volume'] / quotes_df['weightedAverageShsOut'] * quotes_df['vwap']
  mean_vol = quotes_df[['symbol', 'vol_daily']].groupby('symbol').rolling(period, min_periods=2).mean().reset_index()[['symbol', 'vol_daily']]
  return mean_vol['vol_daily']

def DAVOL(quotes_df, period_short=20, period_long=120):
  return VOL(quotes_df, period=20) / VOL(quotes_df, period=120)

def retained_earnings_per_share(quotes_df):
  return quotes_df['retainedEarnings'] / quotes_df['weightedAverageShsOut']

def cash_flow_to_price_ratio(quotes_df):
  return quotes_df['freeCashFlowPerShare'] / quotes_df['close']

def financial_liability_to_price(quotes_df):
  # financial_liability
  return quotes_df['longTermDebt'] / quotes_df['weightedAverageShsOut'] / quotes_df['close']

def ACCA(quotes_df):
  return quotes_df['freeCashFlowPerShare'] / quotes_df['p_b_ratio_daily_alter']

def sales_to_price_daily(quotes_df):
  book = quotes_df['revenue'] / quotes_df['weightedAverageShsOut']
  price = quotes_df['close']
  return book / price

def earnings_to_price_ratio(quotes_df):
  book = quotes_df['incomeBeforeTax'] / quotes_df['weightedAverageShsOut']
  price = quotes_df['close']
  return book / price

def calculate_rank(quotes_df, column_to_rank, group_by_columns, ascending=False, method='max'):
  return quotes_df.groupby(group_by_columns)[column_to_rank].rank(ascending=ascending, method=method)


def google_trends(quotes_df):
  # скачивает с гугл трендс историю поиска по ключевым словам заданным ниже
  pytrends = TrendReq(hl='en-US', tz=360, timeout=(9,9))

  kw_list = ["debt", 'color', 'stocks', 'restaurant', 'portfolio', 'inflation', 'housing', 'ring',
            'crisis financial', 'stock crash','labour', 'stock market','buy and hold', 'unemployment',
            ]

#              'economics', 'credit', 'markets', 'return',  'money', 'religion', 'cancer', 'growth',
#             'investment', 'hedge', 'marriage', 'bonds', 'derivatives', 'headlines', 'profit', 'society', 
#              'leverage', 'loss', 'cash', 'office', 'fine',  'banking', 'crisis', 'happy', 'car',
#             'nasdaq', 'gains', 'finance', 'sell', 'invest', 'fed', 'house', 'metals', 'travel', 'returns', 'gain', 'default', 'present', 'holiday', 'water', 'rich', 'risk', 'gold', 'success', 'oil', 'war',
#             'economy', 'dow jones', 'chance', 'short sell', 'lifestyle', 'greed', 'food', 'financial market', 'movie', 'nyse', 'ore', 'opporunity', 'health', 'short selling', 'earnings', 'arts',
#             'culture', 'bubble', 'buy', 'trader', 'rare earths', 'tourism', 'politics', 'energy', 'consume', 'consumtion', 'freedom', 'dividend', 'world', 'conflict', 'kitchen','forex', 'home', 'crash', 'transaction',
#             'garden', 'fond', 'train', 'fun', 'environment',  ]

  google_trends_data = []

  def request_data(request_d):
    try:
      pytrends.build_payload([request_d], cat=0, timeframe='all', geo='US', gprop='') 
      google_trends_data.append(pytrends.interest_over_time())
    except:
      time.sleep(1)
      request_data(request_d)

  for request in kw_list:
    try:
      request_data(request)
    except:
      print('skip request {}'.format(request))
      pass

  trends_df = reduce(lambda x, y: pd.merge(x, y, on = 'date'), google_trends_data)
  try:
    trends_df = trends_df.drop(['isPartial'], axis=1)
  except:
    pass
  try:
    trends_df = trends_df.drop(['isPartial_x', 'isPartial_y'], axis=1)
  except:
    pass

  trends_df = trends_df.pct_change(periods=3)
  cols = trends_df.columns
  trends_df = trends_df.reset_index() # считаем изменение за месяц по гугл трендс!
  dates = pd.DataFrame(pd.date_range(start='1/1/2004', end='20/05/2021'))
  dates.columns = ['date']
  dates = dates.merge(trends_df, how='left', on='date').fillna(method='ffill')
  return cols, quotes_df.merge(dates, on='date', how='left')


def fill_shares_number(quotes_df):
  quotes_df.loc[quotes_df['weightedAverageShsOut']==0, 'weightedAverageShsOut'] = round(quotes_df['revenue'] / quotes_df['revenuePerShare'],0)
  return quotes_df


def set_ff_groups_median(quotes_df):
  # устанавливает группы акций по 3 факторной модели Ferma-French 
  # на основе этих групп можно будет расчитать историческое изменение цен по этим группам
  market_cap_median = quotes_df['market_cap_daily'].median()
  bp_median = quotes_df['bookValuePerShare'].median()
  profit_median = quotes_df['LAST_YEAR_PERFORMANCE'].median()

  mask_cap_small = quotes_df['market_cap_daily'] < market_cap_median
  mask_cap_big = quotes_df['market_cap_daily'] >= market_cap_median
  mask_book_low = quotes_df['bookValuePerShare'] < bp_median
  mask_book_high = quotes_df['bookValuePerShare'] >= bp_median
  mask_profit_high = quotes_df['LAST_YEAR_PERFORMANCE'] >= profit_median
  mask_profit_low = quotes_df['LAST_YEAR_PERFORMANCE'] < profit_median

  # Book & Cap size
  quotes_df['ff_groups_book_median'] = None
  quotes_df['ff_groups_book_median'].loc[mask_cap_small&mask_book_high] = 'sh'
  quotes_df['ff_groups_book_median'].loc[mask_cap_big&mask_book_high] = 'bh'
  quotes_df['ff_groups_book_median'].loc[mask_cap_small&mask_book_low] = 'sl'
  quotes_df['ff_groups_book_median'].loc[mask_cap_big&mask_book_low] = 'bl'
  quotes_df['ff_groups_book_median'] = quotes_df['ff_groups_book_median'].astype('category')

  # Profit & Cap size
  quotes_df['ff_groups_profit_median'] = None
  quotes_df['ff_groups_profit_median'].loc[mask_cap_small&mask_profit_high] = 'sr'
  quotes_df['ff_groups_profit_median'].loc[mask_cap_big&mask_profit_high] = 'br'
  quotes_df['ff_groups_profit_median'].loc[mask_cap_small&mask_profit_low] = 'sw'
  quotes_df['ff_groups_profit_median'].loc[mask_cap_big&mask_profit_low] = 'bw'
  quotes_df['ff_groups_profit_median'] = quotes_df['ff_groups_profit_median'].astype('category')
  
  return quotes_df[['ff_groups_book_median', 'ff_groups_profit_median']]
  # return quotes_df



def calculate_ff_group_returns(quotes_df):
  quotes_df[['market_cap_daily', 'daily_pct_change']] = quotes_df[['market_cap_daily', 'daily_pct_change']].fillna(0)
  quotes_df.replace([np.inf, -np.inf], np.nan, inplace=True)

  ff_columns = ['ff_groups_book_median', 'ff_groups_profit_median']
  periods = [251, 60, 20]

  for ff_column in ff_columns:

    wm = lambda x: np.average(x, weights=quotes_df.loc[x.index, "market_cap_daily"])
    tmp = quotes_df[['date', ff_column, 'market_cap_daily', 'daily_pct_change']].dropna().groupby(['date', ff_column]).agg(total_cap_ff_group=("market_cap_daily", "sum"),  
                                                        daily_pct_change_weighted_by_cap_mean=("daily_pct_change", wm))

    tmp = tmp.reset_index()
    tmp.rename(columns={'total_cap_ff_group': 'total_cap_{}'.format(ff_column),
                        'daily_pct_change_weighted_by_cap_mean': 'daily_pct_change_weighted_by_cap_{}'.format(ff_column)}, inplace=True) 
    
    tmp.set_index('date', drop=True, inplace=True)
    quotes_df = quotes_df.merge(tmp, how='left', on=['date', ff_column])
    
    for period in periods:

      group_returns = tmp[[ff_column, 'daily_pct_change_weighted_by_cap_{}'.format(ff_column)]].groupby(ff_column).rolling(period, min_periods=2).apply(total_return_from_returns)
      group_returns = group_returns.reset_index(level=0)
      group_returns.rename(columns={'daily_pct_change_weighted_by_cap_{}'.format(ff_column): 'pct_change_{}_{}'.format(ff_column, period)}, inplace=True)

      quotes_df = quotes_df.merge(group_returns, how='left', on=['date', ff_column])

      tmp2 = group_returns.pivot_table(columns=ff_column, values='pct_change_{}_{}'.format(ff_column, period), index='date')
      tmp2.columns = tmp2.columns.astype(str)
      tmp2 = tmp2.reset_index()

      if period == 251: # эти параметры считаем только для годовых значений

        if ff_column == 'ff_groups_book_median':
          tmp2['HML'] = (tmp2['bh'] + tmp2['sh'])/2 - (tmp2['sl'] + tmp2['bl'])/2
          tmp2['SMB'] = (tmp2['sl'] + tmp2['sh'])/2  - (tmp2['bl'] + tmp2['bh'])/2 
          cols_to_merge = ['date', 'bh', 'bl', 'sh', 'sl', 'HML', 'SMB']
          quotes_df = quotes_df.merge(tmp2[cols_to_merge], how='left', on='date')
        
        elif ff_column == 'ff_groups_profit_median':
          tmp2['RMW'] = (tmp2['br'] + tmp2['sr'])/2 - (tmp2['sw'] + tmp2['bw'])/2
          cols_to_merge = ['date', 'sr', 'br', 'sw', 'bw', 'RMW']
          quotes_df = quotes_df.merge(tmp2[cols_to_merge], how='left', on='date')

  return quotes_df

def load_quandl(quotes_df):

    dailySeries = ['LBMA/GOLD']

    for timeSeries in dailySeries:
        try:
            df = quandl.get(timeSeries, column_index='1')
            df.columns = [timeSeries]
            df[timeSeries+'PctDaily'] = df[timeSeries].pct_change()
            df[timeSeries+'60dChange'] = df[timeSeries+'PctDaily'].rolling(60, min_periods=2).apply(total_return_from_returns)
            df[timeSeries+'20dChange'] = df[timeSeries+'PctDaily'].rolling(20, min_periods=2).apply(total_return_from_returns)
            df = df.reset_index().rename(columns={'Date': 'date'})
            df = df.drop(timeSeries+'PctDaily', axis=1)
            quotes_df = quotes_df.merge(df, on='date', how='left')
            
        except:
            pass
            
    return quotes_df
    

def load_FRED_data(quotes_df):
  # T10YFF = 10-Year Treasury Constant Maturity Minus Federal Funds Rate = дни \ на большом датасете можно будет оставить как есть, добавить изменения
  # T10Y2Y = 10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity = дни \ на большом датасете можно будет оставить как есть, добавить изменения
  # BAMLCC7A01015YTRIV = ICE BofA 10-15 Year US Corporate Index Total Return Index Value = индекс облигаций, дни, надо к изменениям переходить
  # FEDFUNDS = FED Funds Rate = к изменениям переходить \ месяц
  # DBAA = Moody's Seasoned Baa Corporate Bond Yield = день, перейти к изменениям
  # NASDAQCOM = NASDAQ / дни
  # SP500 / дни
  # PMETAINDEXM -  Global price of Metal index / month
  # DCOILWTICO - Crude Oil Prices: West Texas Intermediate (WTI) - Cushing, Oklahoma / day
  # DCOILBRENTEU Crude Oil Prices: Brent - Europe/day
  # GOLDAMGBD228NLBM - Gold Fixing Price 10:30 A.M. (London time) in London Bullion Market, based in U.S. Dollars /day
  # DHHNGSP - Henry Hub Natural Gas Spot Price
  # DJFUELUSGULF - Kerosene-Type Jet Fuel Prices: U.S. Gulf Coast
  # DGASUSGULF - Conventional Gasoline Prices: U.S. Gulf Coast, Regular
  # PCOPPUSDM - copper / mont
  # PALLFNFINDEXM - Global Price Index of All Commodities /month
  # PIORECRUSDM - iron ore /month
  # PWHEAMTUSDM - wheat / month
  # PSOYBUSDM - soybeans / month

  # daily data
  dailySeries = ['T10YFF', 'T10Y2Y', 'BAMLCC7A01015YTRIV', 'DBAA', 'NASDAQCOM', 
                 'SP500', 'DCOILWTICO', 'DCOILBRENTEU',  
                 'DHHNGSP', 'DJFUELUSGULF', 'DGASUSGULF']

  for timeSeries in dailySeries: 
        try:
            df = pd.DataFrame(fred.get_series(timeSeries, observation_start='1962-01-01', observation_end=datetime.date.today()))
            df.columns = [timeSeries]
            df[timeSeries+'PctDaily'] = df[timeSeries].pct_change()
            df[timeSeries+'60dChange'] = df[timeSeries+'PctDaily'].rolling(60, min_periods=2).apply(total_return_from_returns)
            df[timeSeries+'20dChange'] = df[timeSeries+'PctDaily'].rolling(20, min_periods=2).apply(total_return_from_returns)
            df['date'] = df.index
            df = df.drop(timeSeries+'PctDaily', axis=1)
            quotes_df = quotes_df.merge(df, on='date', how='left')
        except:
            print('skip', timeSeries)

  # monthly data
  for timeSeries in ['FEDFUNDS', 'PMETAINDEXM', 'PCOPPUSDM', 'PALLFNFINDEXM', 'PIORECRUSDM', 'PWHEAMTUSDM', 'PSOYBUSDM']:
        try:
            df_series = pd.DataFrame(fred.get_series(timeSeries, frequency = 'm', observation_start='1962-01-01', observation_end=datetime.date.today()))
            df_series.index = df_series.index.to_period('M').to_timestamp('M')
            df_series.columns = [timeSeries]
            df_series[timeSeries+'PctMonth'] = df_series[timeSeries].pct_change()
            df_series[timeSeries+'Pct3Month'] = df_series[timeSeries+'PctMonth'].rolling(3, min_periods=2).apply(total_return_from_returns)
            df_series[timeSeries+'Pct6Month'] = df_series[timeSeries+'PctMonth'].rolling(6, min_periods=2).apply(total_return_from_returns)
            df_series['date'] = df_series.index

            df = pd.DataFrame(pd.date_range(start='1962-01-01', end=datetime.date.today()))
            df.columns = ['date']
            df = df.merge(df_series, how='left', on='date')
            df = df.fillna(method='ffill')
            quotes_df = quotes_df.merge(df, on='date', how='left')
        except:
            print('skip', timeSeries)

  return reduce_mem_usage(quotes_df)



In [ ]:
# Подготовка таблицы с данными по котировкам для расчета индикаторов
with open('{}historical_price_last_year.pickle'.format(DIRECTORY), 'rb') as f: 
  quotes = pickle.load(f)[['date', 'symbol', 'volume', 'close', 'vwap']]

quotes = quotes.drop_duplicates(subset=['date', 'symbol'])
quotes = quotes[quotes['close']>0]
quotes = quotes.sort_values(by=['symbol', 'date']) # теперь старые даты сверху
quotes = quotes.reset_index(drop=True)
quotes['daily_pct_change'] = daily_returns(quotes[['symbol', 'close']]) # добавляем дневное изменение расчитанное по клоузам, т.к. API дает дневное изменение как (C-O)/O 
quotes = next_month_return_and_target_alt(quotes) # добавляет изменение % за след. календарный месяц, отмечает категории таргета
quotes = business_days(quotes) # отмечаем последние дни торгового месяца - будет использоваться для формирования прогноза на эти даты
quotes = reduce_mem_usage(quotes)

In [ ]:
# загружаем pl, bs, fin growth, key ratios
quotes = load_fundamentals(quotes) # добавляем фундаментальные данные
quotes = reduce_mem_usage(quotes)

# поскольку смердженные фундаменталки содержат кучу na - делаем ffill чтобы их заполнить
fund_cols = ['period_end','fillingDate', 'acceptedDate', 'ebitdaratio', 'operatingIncomeRatio', 'incomeBeforeTaxRatio',
 'netIncomeRatio', 'eps', 'weightedAverageShsOut', 'ebitda', 'incomeBeforeTax','interestExpense',
 'revenue', 'revenueGrowth', 'grossProfitGrowth', 'ebitgrowth','operatingIncomeGrowth', 'netIncomeGrowth',
 'epsgrowth', 'epsdilutedGrowth', 'weightedAverageSharesGrowth', 'weightedAverageSharesDilutedGrowth',
 'dividendsperShareGrowth','operatingCashFlowGrowth', 'freeCashFlowGrowth', 'tenYRevenueGrowthPerShare', 'fiveYRevenueGrowthPerShare',
 'threeYRevenueGrowthPerShare','tenYOperatingCFGrowthPerShare', 'fiveYOperatingCFGrowthPerShare', 'threeYOperatingCFGrowthPerShare',
 'tenYNetIncomeGrowthPerShare', 'fiveYNetIncomeGrowthPerShare', 'threeYNetIncomeGrowthPerShare', 'tenYShareholdersEquityGrowthPerShare',
 'fiveYShareholdersEquityGrowthPerShare', 'threeYShareholdersEquityGrowthPerShare','tenYDividendperShareGrowthPerShare',
 'fiveYDividendperShareGrowthPerShare', 'threeYDividendperShareGrowthPerShare', 'receivablesGrowth', 'inventoryGrowth',
 'assetGrowth', 'bookValueperShareGrowth', 'debtGrowth', 'rdexpenseGrowth', 'sgaexpensesGrowth', 'revenuePerShare',
 'netIncomePerShare', 'operatingCashFlowPerShare', 'freeCashFlowPerShare', 'cashPerShare', 'bookValuePerShare',
 'tangibleBookValuePerShare', 'shareholdersEquityPerShare', 'interestDebtPerShare', 'marketCap', 'enterpriseValue', 'peRatio',
 'priceToSalesRatio', 'pocfratio', 'pfcfRatio', 'pbRatio', 'ptbRatio', 'evToSales', 'enterpriseValueOverEBITDA',
 'evToOperatingCashFlow', 'evToFreeCashFlow', 'earningsYield', 'freeCashFlowYield','debtToEquity', 'debtToAssets', 'netDebtToEBITDA',
 'currentRatio', 'interestCoverage', 'incomeQuality', 'dividendYield', 'payoutRatio', 'salesGeneralAndAdministrativeToRevenue',
 'researchAndDdevelopementToRevenue', 'intangiblesToTotalAssets', 'capexToOperatingCashFlow', 'capexToRevenue',
 'capexToDepreciation', 'stockBasedCompensationToRevenue', 'grahamNumber', 'roic', 'returnOnTangibleAssets', 'grahamNetNet',
 'workingCapital', 'tangibleAssetValue', 'netCurrentAssetValue', 'investedCapital', 'averageReceivables', 'averagePayables',
 'averageInventory', 'daysSalesOutstanding', 'daysPayablesOutstanding', 'daysOfInventoryOnHand', 'receivablesTurnover',
 'payablesTurnover', 'inventoryTurnover', 'roe', 'capexPerShare','totalStockholdersEquity','retainedEarnings','longTermDebt']

for col in fund_cols:
    try:
        quotes[col] = quotes[[col, 'symbol']].groupby('symbol').fillna(method='ffill')[col]
    except:
        quotes[col] = quotes[col].astype(float)
        quotes[col] = quotes[[col, 'symbol']].groupby('symbol').fillna(method='ffill')[col]

quotes = reduce_mem_usage(quotes)
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(quotes.memory_usage(index=True).sum())))

In [ ]:
quotes['LAST_YEAR_PERFORMANCE'] = ROC(quotes[['symbol', 'close']], period=251)
quotes = load_profile(quotes) # добавляем данные по профайлу компании
quotes = reduce_mem_usage(quotes)
quotes = fill_shares_number(quotes) # не везде заполнено количество акций, поэтому высчитываем этот показатель на основе выручки и выручки\акция
quotes['p_b_ratio_daily_alter'] = p_b_ratio_daily_alter(quotes[['close', 'bookValuePerShare']])
quotes['net_profit_to_total_operate_revenue_ttm'] = net_profit_to_total_operate_revenue_ttm(quotes[['netIncomePerShare', 'revenuePerShare']])
quotes['market_cap_daily'] = market_cap_daily(quotes[['weightedAverageShsOut', 'close']])
quotes['VOL20'] = VOL(quotes[['symbol', 'volume', 'weightedAverageShsOut', 'vwap']])
quotes['DAVOL20'] = DAVOL(quotes[['symbol', 'volume', 'weightedAverageShsOut', 'vwap']])
quotes['retained_earnings_per_share'] = retained_earnings_per_share(quotes[['retainedEarnings', 'weightedAverageShsOut']])
quotes['cash_flow_to_price_ratio'] = cash_flow_to_price_ratio(quotes[['freeCashFlowPerShare', 'close']])
quotes['financial_liability_to_price'] = financial_liability_to_price(quotes[['longTermDebt', 'close', 'weightedAverageShsOut']])
quotes['ACCA'] = ACCA(quotes[['freeCashFlowPerShare', 'p_b_ratio_daily_alter']])
quotes['Price_index_6m'] = ROC(quotes[['symbol', 'close']], period=int(252/2)) #измненеие цены за последние 6м

In [ ]:
# ###########
# Трехфакторная модель Ferma-French (без рыночной беты пока что)
quotes[['ff_groups_book_median', 'ff_groups_profit_median']] = set_ff_groups_median(quotes) # set farma french 3 factor model groups 
quotes = reduce_mem_usage(quotes)
quotes = calculate_ff_group_returns(quotes)
quotes = reduce_mem_usage(quotes)
quotes['VARIANCE20_ff_groups_profit_median'] = VARIANCE(quotes[['symbol', 'daily_pct_change_weighted_by_cap_ff_groups_profit_median']], column='daily_pct_change_weighted_by_cap_ff_groups_profit_median')
quotes['VARIANCE20_ff_groups_book_median'] = VARIANCE(quotes[['symbol', 'daily_pct_change_weighted_by_cap_ff_groups_book_median']], column='daily_pct_change_weighted_by_cap_ff_groups_book_median')
quotes = reduce_mem_usage(quotes)
quotes['diff_ff_group_profit_251_return_and_stock_251_return'] = (quotes['LAST_YEAR_PERFORMANCE'] - quotes['pct_change_ff_groups_profit_median_251']) / quotes['pct_change_ff_groups_profit_median_251']
quotes['diff_ff_group_book_251_return_and_stock_251_return'] = (quotes['LAST_YEAR_PERFORMANCE'] - quotes['pct_change_ff_groups_book_median_251']) / quotes['pct_change_ff_groups_book_median_251']
quotes = reduce_mem_usage(quotes)

## quantum fundamental factors
quotes['sales_to_price_daily'] = sales_to_price_daily(quotes[['revenue', 'weightedAverageShsOut', 'close']])
quotes['earnings_to_price_daily'] = earnings_to_price_ratio(quotes[['incomeBeforeTax', 'weightedAverageShsOut', 'close']])
quotes = reduce_mem_usage(quotes)
## Ranking
quotes['rank_pb_value'] = calculate_rank(quotes[['date', 'p_b_ratio_daily_alter']], column_to_rank='p_b_ratio_daily_alter', group_by_columns='date')
quotes['rank_Price_index_6m'] = calculate_rank(quotes[['date', 'Price_index_6m']], column_to_rank='Price_index_6m', group_by_columns='date')
quotes['rank_dividendYield'] = calculate_rank(quotes[['date', 'dividendYield']], column_to_rank='dividendYield', group_by_columns='date')
quotes['rank_enterpriseValueOverEBITDA'] = calculate_rank(quotes[['date', 'enterpriseValueOverEBITDA']], column_to_rank='enterpriseValueOverEBITDA', group_by_columns='date')
quotes['rank_sales_to_price_daily'] = calculate_rank(quotes[['date', 'sales_to_price_daily']], column_to_rank='sales_to_price_daily', group_by_columns='date')
quotes['rank_earnings_to_price_daily'] = calculate_rank(quotes[['date', 'earnings_to_price_daily']], column_to_rank='earnings_to_price_daily', group_by_columns='date')
quotes = reduce_mem_usage(quotes)
## total rank stratagies
## priceindex 6m + booktoprice
quotes['rank_first_strategie_value'] = quotes['rank_pb_value'] + quotes['rank_Price_index_6m']
quotes['rank_first_strategie'] = calculate_rank(quotes[['date', 'rank_first_strategie_value']], column_to_rank='rank_first_strategie_value', group_by_columns='date')
quotes = reduce_mem_usage(quotes)
# ## p/b+ p/s + ebitda/ev + p/e + div yield
# # OShaughnessy trending value investment strategy
# # https://www.quant-investing.com/strategies/oshaughnessy-trending-value-investment-strategy
# # Тут не хватает Price to cash flow!!!
quotes['rank_second_strategie_value'] = quotes['rank_pb_value'] + quotes['rank_sales_to_price_daily'] + quotes['rank_enterpriseValueOverEBITDA'] + quotes['rank_earnings_to_price_daily'] + quotes['rank_dividendYield']
quotes['rank_second_strategie'] = calculate_rank(quotes[['date', 'rank_second_strategie_value']], column_to_rank='rank_second_strategie_value', group_by_columns='date')
quotes = reduce_mem_usage(quotes)
#####################

# TO DO:
# 1) Qi Value Investment Strategy: EBITDA Yield + FCF Yield + Liquidity (Q.i) is calculated as Adjusted Profits / Yearly trading value.
# https://www.quant-investing.com/strategies/qi-value-investment-strategy

# 2) Quality Adjusted Value Microcap Investment Strategy - OShaughnessy
# https://www.quant-investing.com/strategies/quality-adjusted-value-microcap-investment-strategy---oshaughnessy
# o do this they removed companies that fell in the bottom 10% of the following five ratios or indicators:
# Change in Net Operating Assets (NOA = Operating Assets - Operating Liabilities)
# Debt to Equity Ratio
# 1-Year change in debt
# Return on Invested Capital (ROIC)
# Free cash flow yield
# The portfolios were re-balanced yearly and the portfolios were equal-weighted.

# 3) Free cash flow yield and Price Index 12 months investment strategy
# 4) https://www.quant-investing.com/strategies/neglected-value-and-momentum---europe

# https://virtusinterpress.org/IMG/pdf/cocv15i1art4.pdf
# We used the same market indicators of Tsuji (2012): inflation, credit risk, 
# dividend yield and volatility of market return. The inflation was measured 
# through Consumer Price Index (IPCA). The credit risk was calculated through 
# the difference between the working capital interest rate and the interbank 
# market interest rate (Schor et al., 2002). The market dividend yield is 
# the average dividend yield (dividend of the last 12 months divided by the 
# share price) of the companies in the sample weighted by their market value. 
# Market volatility was calculated by the one-year standard deviation of the 
# stock market index IBrX-100, and the country risk was measured by EMBI + Brazil index.
#  We also add in the regressions an overall corporate governance measure, 
#  proxied by the percentage of companies listed on “Novo Mercado” (New Market).

# 5) Earnings estimates count - как много аналитиков покрывают компанию, больше - лучше
# 6) N-month change in recommendation - This factor ranks stocks by the change in consensus
# recommendation over the prior N month, where improvements are desirable (regardless of whether
# they have moved from strong sell to sell or buy to strong buy and so on).
# 7)12-month change in shares outstanding This factor measures the change in a company's 
# split-adjusted share count over the last 12 months, where a negative change implies
# share buybacks and is desirable because it signals that management views the stock
# as cheap relative to its intrinsic and, hence, future value.
# 8) The metric tracks the 6-month change in mean analyst target price. A higher positive change is naturally more desirable.
# 9) net earnings revisions - This factor expresses the difference between upward and downward revisions to earnings estimates as a percentage of the total number of revisions.
# 10) Short interest to shares outstanding - This measure is the percentage of shares
# outstanding currently being sold short, that is, sold by an investor who ha
# borrowed the share and needs to repurchase it at a later day while speculating that 
# its price will fall. Hence, a high level of short interest indicates negative sentiment 
# and is expected to signal poor performance going forward.
# 11) The price/earnings to growth (PEG) ratio divides a stock's price-to- earnings (P/E) 
# ratio by the earnings growth rate for a given period. The ratio adjusts the price paid for a 
# dollar of earnings (measured by the P/E ratio) by the company's earnings growth.
# 12) Earnings yield 1- year forward
# 13) higher total accruals relative to assets make low earnings quality more likely
# 14) asset turnover dividing sales by total assets. A higher turnover is better.
# 15) asset turnover 12month change
# 16) payout ratio - The share of earnings paid out in dividends to shareholders. Stocks with higher payout ratios are ranked higher.
# 17) 101 alpha factors to use: https://arxiv.org/pdf/1601.00991.pdf



In [ ]:
# Технические индикаторы

quotes['VOSC'] = VOSC(quotes[['symbol', 'volume']])
quotes['VMACD'] = VMACD2(quotes[['symbol', 'volume', 'date']])
quotes['ROC'] = ROC(quotes[['symbol', 'close']])
quotes['VOLUME1M'] = VOLUME(quotes[['symbol', 'volume', 'ROC']])
quotes['Price1M'] = Price1M(quotes[['symbol', 'close']])
quotes['MAC20'] = MAC(quotes[['symbol', 'close']])
quotes['VARIANCE20'] = VARIANCE(quotes[['symbol', 'daily_pct_change']])
quotes['KURTOSIS20'] = KURTOSIS(quotes[['symbol', 'close']])
quotes['SKEW20'] = SKEW(quotes[['symbol', 'close']])
quotes['TRIX10'] = TRIX(quotes[['symbol', 'close']])
# # quotes['MFI14'] = MFI(quotes[['symbol', 'close', 'low', 'high', 'volume']]) # очень долго считается!
quotes['BOLL_DOWN20'] = boll_down(quotes[['symbol', 'close']])
quotes['BOLL_UP20'] = boll_up(quotes[['symbol', 'close']])
quotes['SHARP20'] = SHARPE_RATIO(quotes[['symbol', 'close']], period=20)
quotes['SHARP60'] = SHARPE_RATIO(quotes[['symbol', 'close']], period=60)
quotes = reduce_mem_usage(quotes)

# to do:
# 1) RSI
# 2) 12-month price momentum volume adjustment. The indicator normalizes the total return over the previous 12 months by volume dividing it by the standard deviation of these returns.
# 3) Price acceleration calculates the gradient of the price trend (adjusted for volatility) 
# using linear regression on daily prices for a longer and a shorter period, 
# for example, 1 year and 3 months of trading days, and compares the change in the slope as a measure of price acceleration.
# 4) Percent off 52-week high
# 5) use talib lib! 


In [ ]:
# # # Alternative DATA
words, quotes = google_trends(quotes) # добавлем историю поиска по гугл трендс
quotes = reduce_mem_usage(quotes)
quotes = load_FRED_data(quotes) # см список загружаемых данных по макро с сайта FRED внутри функции
quotes = reduce_mem_usage(quotes)
quotes = load_quandl(quotes) # подгружаем с квандла золото, потому что фред не хотел отдавать золото по api
quotes = reduce_mem_usage(quotes)

In [ ]:
# Сохраняем датасет по кускам

base_cols = ['date','symbol','volume', 'close', 'vwap','daily_pct_change','year',
 'month','price_change_next_calendar_month','target','last_bday_current_month','is_last_bday_current_month',]

fund_cols1 = ['date','symbol','period_end','fillingDate', 'acceptedDate',
 'ebitdaratio', 'operatingIncomeRatio', 'incomeBeforeTaxRatio', 'netIncomeRatio', 'eps', 'weightedAverageShsOut', 'ebitda',
 'incomeBeforeTax','interestExpense', 'revenue', 'revenueGrowth', 'grossProfitGrowth', 'ebitgrowth', 'operatingIncomeGrowth',
 'netIncomeGrowth', 'epsgrowth', 'epsdilutedGrowth', 'weightedAverageSharesGrowth', 'weightedAverageSharesDilutedGrowth',
 'dividendsperShareGrowth', 'operatingCashFlowGrowth', 'freeCashFlowGrowth', 'tenYRevenueGrowthPerShare','fiveYRevenueGrowthPerShare',
 'threeYRevenueGrowthPerShare', 'tenYOperatingCFGrowthPerShare','fiveYOperatingCFGrowthPerShare','threeYOperatingCFGrowthPerShare',]

fund_cols2 =['date','symbol',
     'tenYNetIncomeGrowthPerShare','fiveYNetIncomeGrowthPerShare', 'threeYNetIncomeGrowthPerShare',
 'tenYShareholdersEquityGrowthPerShare', 'fiveYShareholdersEquityGrowthPerShare', 'threeYShareholdersEquityGrowthPerShare',
 'tenYDividendperShareGrowthPerShare', 'fiveYDividendperShareGrowthPerShare', 'threeYDividendperShareGrowthPerShare',
 'receivablesGrowth', 'inventoryGrowth', 'assetGrowth', 'bookValueperShareGrowth', 'debtGrowth', 'rdexpenseGrowth',
 'sgaexpensesGrowth', 'revenuePerShare','netIncomePerShare', 'operatingCashFlowPerShare', 'freeCashFlowPerShare',
 'cashPerShare', 'bookValuePerShare', 'tangibleBookValuePerShare', 'shareholdersEquityPerShare', 'interestDebtPerShare',
 'marketCap', 'enterpriseValue', 'peRatio', 'priceToSalesRatio', 'pocfratio', 'pfcfRatio', 'pbRatio', 'ptbRatio', 'evToSales',
]

fund_cols3 = ['date','symbol',
    'enterpriseValueOverEBITDA', 'evToOperatingCashFlow','evToFreeCashFlow',
 'earningsYield', 'freeCashFlowYield', 'debtToEquity', 'debtToAssets','netDebtToEBITDA', 'currentRatio',
 'interestCoverage', 'incomeQuality', 'dividendYield', 'payoutRatio','salesGeneralAndAdministrativeToRevenue',
 'researchAndDdevelopementToRevenue', 'intangiblesToTotalAssets', 'capexToOperatingCashFlow', 'capexToRevenue', 'capexToDepreciation',
 'stockBasedCompensationToRevenue', 'grahamNumber', 'roic', 'returnOnTangibleAssets', 'grahamNetNet', 'workingCapital',
 'tangibleAssetValue', 'netCurrentAssetValue', 'investedCapital','averageReceivables', 'averagePayables',
 'averageInventory', 'daysSalesOutstanding', 'daysPayablesOutstanding', 'daysOfInventoryOnHand', 'receivablesTurnover',
 'payablesTurnover', 'inventoryTurnover', 'roe', 'capexPerShare', 'totalStockholdersEquity', 'retainedEarnings', 'longTermDebt'
]

profile_cols = ['date','symbol','industry','ceo','sector', 'fullTimeEmployees','isActivelyTrading','isEtf',]

quant_cols = ['date','symbol','p_b_ratio_daily_alter',
 'net_profit_to_total_operate_revenue_ttm', 'market_cap_daily','retained_earnings_per_share',
 'cash_flow_to_price_ratio', 'financial_liability_to_price','sales_to_price_daily',
 'earnings_to_price_daily', 'rank_pb_value', 'Price_index_6m',
 'rank_Price_index_6m', 'rank_dividendYield', 'rank_enterpriseValueOverEBITDA',
 'rank_sales_to_price_daily', 'rank_earnings_to_price_daily', 'rank_first_strategie_value',
 'rank_first_strategie', 'rank_second_strategie_value','rank_second_strategie',]

ff_cols = ['date','symbol','ff_groups_book_median',
 'ff_groups_profit_median', 'total_cap_ff_groups_book_median', 'daily_pct_change_weighted_by_cap_ff_groups_book_median',
 'pct_change_ff_groups_book_median_251', 'bh', 'bl', 'sh', 'sl', 'HML', 'SMB',
 'pct_change_ff_groups_book_median_60', 'pct_change_ff_groups_book_median_20', 'total_cap_ff_groups_profit_median',
 'daily_pct_change_weighted_by_cap_ff_groups_profit_median', 'pct_change_ff_groups_profit_median_251', 'sr', 'br', 'sw', 'bw','RMW', 'pct_change_ff_groups_profit_median_60',
 'pct_change_ff_groups_profit_median_20', 'VARIANCE20_ff_groups_profit_median','VARIANCE20_ff_groups_book_median',
 'diff_ff_group_profit_251_return_and_stock_251_return','diff_ff_group_book_251_return_and_stock_251_return',]

indeicators_cols = ['date','symbol', 'LAST_YEAR_PERFORMANCE','VOL20','DAVOL20', 'ACCA','VOSC',
 'VMACD', 'ROC', 'VOLUME1M', 'Price1M', 'MAC20', 'VARIANCE20', 'KURTOSIS20', 'SKEW20',
 'TRIX10', 'BOLL_DOWN20', 'BOLL_UP20', 'SHARP20', 'SHARP60']

google_cols = ['date','symbol', 'debt', 'color', 'stocks', 'restaurant', 'portfolio', 'inflation', 'housing',
 'ring', 'crisis financial', 'stock crash', 'labour', 'stock market', 'buy and hold','unemployment']

fred_cols = ['date', 'symbol', 'T10YFF',
       'T10YFF60dChange', 'T10YFF20dChange', 'T10Y2Y', 'T10Y2Y60dChange',
       'T10Y2Y20dChange', 'BAMLCC7A01015YTRIV', 'BAMLCC7A01015YTRIV60dChange',
       'BAMLCC7A01015YTRIV20dChange', 'DBAA', 'DBAA60dChange', 'DBAA20dChange',
       'NASDAQCOM', 'NASDAQCOM60dChange', 'NASDAQCOM20dChange', 'SP500',
       'SP50060dChange', 'SP50020dChange', 'DCOILWTICO', 'DCOILWTICO60dChange',
       'DCOILWTICO20dChange', 'DCOILBRENTEU', 'DCOILBRENTEU60dChange',
       'DCOILBRENTEU20dChange', 'DHHNGSP', 'DHHNGSP60dChange',
       'DHHNGSP20dChange', 'DJFUELUSGULF', 'DJFUELUSGULF60dChange',
       'DJFUELUSGULF20dChange', 'DGASUSGULF', 'DGASUSGULF60dChange',
       'DGASUSGULF20dChange', 'FEDFUNDS', 'FEDFUNDSPctMonth',
       'FEDFUNDSPct3Month', 'FEDFUNDSPct6Month', 'PMETAINDEXM',
       'PMETAINDEXMPctMonth', 'PMETAINDEXMPct3Month', 'PMETAINDEXMPct6Month',
       'PCOPPUSDM', 'PCOPPUSDMPctMonth', 'PCOPPUSDMPct3Month',
       'PCOPPUSDMPct6Month', 'PALLFNFINDEXM', 'PALLFNFINDEXMPctMonth',
       'PALLFNFINDEXMPct3Month', 'PALLFNFINDEXMPct6Month', 'PIORECRUSDM',
       'PIORECRUSDMPctMonth', 'PIORECRUSDMPct3Month', 'PIORECRUSDMPct6Month',
       'PWHEAMTUSDM', 'PWHEAMTUSDMPctMonth', 'PWHEAMTUSDMPct3Month',
       'PWHEAMTUSDMPct6Month', 'PSOYBUSDM', 'PSOYBUSDMPctMonth',
       'PSOYBUSDMPct3Month', 'PSOYBUSDMPct6Month']

quandl_cols = ['date', 'symbol', 'LBMA/GOLD',
       'LBMA/GOLD60dChange', 'LBMA/GOLD20dChange']

# сохраняем датасет  кусками
with open('{}base_df.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(quotes[base_cols], f)

with open('{}fundamental1_df.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(quotes[fund_cols1], f)

with open('{}fundamental2_df.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(quotes[fund_cols2], f)

with open('{}fundamental3_df.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(quotes[fund_cols3], f)

with open('{}profile_df.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(quotes[profile_cols], f)

with open('{}quant_df.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(quotes[quant_cols], f)

with open('{}ff_df.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(quotes[ff_cols], f)

with open('{}indicators_df.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(quotes[indeicators_cols], f)

with open('{}google_df.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(quotes[google_cols], f)

with open('{}fred_df.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(quotes[fred_cols], f)

with open('{}quandl_df.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(quotes[quandl_cols], f)

del quotes

In [ ]:
### Объединение датасета в одну таблицу и фильтр только последних дней месяца - только в них будет торговля, все остальные нам не нужны

df_names = ['fundamental1_df', 'fundamental2_df',
'fundamental3_df','profile_df','quant_df','ff_df','indicators_df','google_df','fred_df','quandl_df',]
skip_cols = ['date', 'symbol']

with open('{}base_df.pickle'.format(DIRECTORY), 'rb') as f: 
  quotes = pickle.load(f)

mask_end_month = quotes['is_last_bday_current_month']==True
quotes = quotes[mask_end_month]

for df_name in df_names:
    with open('{}{}.pickle'.format(DIRECTORY, df_name), 'rb') as f: 
      df = pickle.load(f)[mask_end_month]
    df = df[list(set(df.columns)-set(skip_cols))]
    quotes = quotes.merge(df, left_index=True, right_index=True)

with open('{}final_df.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(quotes, f)

## ОБУЧЕНИЕ И ТЕСТ МОДЕЛИ

In [3]:
def cal_beta(strategy):
  nasdaq = quotes[['date','NASDAQCOM']].drop_duplicates().sort_values(by=['date']).set_index('date').dropna()
  nasdaq['nasdaq_pct_change'] = nasdaq['NASDAQCOM'].astype(float).pct_change()
  strategy.columns = ['strategy']
  strategy['strategy'] = strategy['strategy'].shift(1)
  strategy['strategy_pct_chage'] = strategy['strategy'].pct_change()
  strategy = strategy.merge(nasdaq, how='left', left_index=True, right_index=True)
  strategy = strategy.dropna()
  model = LinearRegression()
  model.fit(strategy['nasdaq_pct_change'].values.reshape(-1, 1), strategy['strategy_pct_chage'].values.reshape(-1, 1))
  beta = model.coef_[0][0]
  return beta

In [ ]:
# Загружаем финальный датасет для обучения
with open('{}final_df.pickle'.format(DIRECTORY), 'rb') as f: 
  quotes = pickle.load(f)

skip_symbols = quotes[quotes['price_change_next_calendar_month']>5]['symbol'].unique() # скипаем тикеры с глючками в close (поставщик данных прокосячил на парочке)
len(quotes[-quotes['symbol'].isin(skip_symbols)]['symbol'].unique())

In [ ]:
features_columns = [
 'fiveYRevenueGrowthPerShare',
 'threeYRevenueGrowthPerShare',
 'grossProfitGrowth',
 'ebitda',
 'weightedAverageSharesGrowth',
 'tenYRevenueGrowthPerShare',
 'epsgrowth',
 'operatingCashFlowGrowth',
 'tenYOperatingCFGrowthPerShare',
 'freeCashFlowGrowth',
 'ebitgrowth',
 'revenue',
 'operatingIncomeRatio',
 'eps',
 'epsdilutedGrowth',
 'netIncomeGrowth',
 'weightedAverageSharesDilutedGrowth',
 'incomeBeforeTaxRatio',
 'operatingIncomeGrowth',
 'incomeBeforeTax',
 'ebitdaratio',
 'dividendsperShareGrowth',
 'netIncomeRatio',
 'threeYOperatingCFGrowthPerShare',
 'weightedAverageShsOut',
 'revenueGrowth',
 'interestExpense',
 'fiveYOperatingCFGrowthPerShare',
 'rdexpenseGrowth',
 'bookValuePerShare',
 'marketCap',
 'tenYShareholdersEquityGrowthPerShare',
 'fiveYDividendperShareGrowthPerShare',
 'interestDebtPerShare',
 'tenYDividendperShareGrowthPerShare',
 'shareholdersEquityPerShare',
 'inventoryGrowth',
 'tangibleBookValuePerShare',
 'threeYNetIncomeGrowthPerShare',
 'receivablesGrowth',
 'priceToSalesRatio',
 'pfcfRatio',
 'sgaexpensesGrowth',
 'threeYDividendperShareGrowthPerShare',
 'fiveYShareholdersEquityGrowthPerShare',
 'assetGrowth',
 'revenuePerShare',
 'debtGrowth',
 'freeCashFlowPerShare',
 'pocfratio',
 'pbRatio',
 'fiveYNetIncomeGrowthPerShare',
 'threeYShareholdersEquityGrowthPerShare',
 'tenYNetIncomeGrowthPerShare',
 'bookValueperShareGrowth',
 'evToSales',
 'netIncomePerShare',
 'enterpriseValue',
 'operatingCashFlowPerShare',
 'cashPerShare',
 'ptbRatio',
 'peRatio',
 'retainedEarnings',
 'daysSalesOutstanding',
 'debtToEquity',
 'longTermDebt',
 'earningsYield',
 'inventoryTurnover',
 'returnOnTangibleAssets',
 'averagePayables',
 'daysPayablesOutstanding',
 'interestCoverage',
 'freeCashFlowYield',
 'investedCapital',
 'netCurrentAssetValue',
 'salesGeneralAndAdministrativeToRevenue',
 'workingCapital',
 'incomeQuality',
 'currentRatio',
 'enterpriseValueOverEBITDA',
 'averageReceivables',
 'roe',
 'grahamNumber',
 'stockBasedCompensationToRevenue',
 'daysOfInventoryOnHand',
 'evToOperatingCashFlow',
 'dividendYield',
 'payablesTurnover',
 'researchAndDdevelopementToRevenue',
 'tangibleAssetValue',
 'averageInventory',
 'totalStockholdersEquity',
 'capexToRevenue',
 'roic',
 'payoutRatio',
 'capexToOperatingCashFlow',
 'debtToAssets',
 'evToFreeCashFlow',
 'intangiblesToTotalAssets',
 'netDebtToEBITDA',
 'grahamNetNet',
 'capexPerShare',
 'receivablesTurnover',
 'capexToDepreciation',
 'industry',
 'sector',
 'fullTimeEmployees',
 'rank_second_strategie_value',
 'rank_sales_to_price_daily',
 'rank_Price_index_6m',
 'net_profit_to_total_operate_revenue_ttm',
 'cash_flow_to_price_ratio',
 'financial_liability_to_price',
 'rank_dividendYield',
 'p_b_ratio_daily_alter',
 'rank_pb_value',
 'rank_earnings_to_price_daily',
 'market_cap_daily',
 'rank_second_strategie',
 'sales_to_price_daily',
 'rank_first_strategie_value',
 'rank_enterpriseValueOverEBITDA',
 'rank_first_strategie',
 'retained_earnings_per_share',
 'Price_index_6m',
 'earnings_to_price_daily',
 'daily_pct_change_weighted_by_cap_ff_groups_book_median',
 'pct_change_ff_groups_profit_median_20',
 'sl',
 'pct_change_ff_groups_profit_median_251',
 'pct_change_ff_groups_book_median_60',
 'pct_change_ff_groups_book_median_20',
 'ff_groups_profit_median',
 'sr',
 'br',
 'total_cap_ff_groups_book_median',
 'diff_ff_group_book_251_return_and_stock_251_return',
 'pct_change_ff_groups_profit_median_60',
 'HML',
 'bw',
 'diff_ff_group_profit_251_return_and_stock_251_return',
 'RMW',
 'daily_pct_change_weighted_by_cap_ff_groups_profit_median',
 'sw',
 'bh',
 'SMB',
 'ff_groups_book_median',
 'VARIANCE20_ff_groups_profit_median',
 'pct_change_ff_groups_book_median_251',
 'bl',
 'sh',
 'VARIANCE20_ff_groups_book_median',
 'total_cap_ff_groups_profit_median',
 'VOSC',
 'VOL20',
 'TRIX10',
 'SHARP20',
 'SHARP60',
 'BOLL_DOWN20',
 'Price1M',
 'BOLL_UP20',
 'VARIANCE20',
 'ROC',
 'VMACD',
 'KURTOSIS20',
 'ACCA',
 'MAC20',
 'DAVOL20',
 'SKEW20',
 'LAST_YEAR_PERFORMANCE',
 'VOLUME1M',
 'DHHNGSP60dChange',
 'DGASUSGULF20dChange',
 'DCOILBRENTEU60dChange',
 'T10Y2Y20dChange',
 'SP50060dChange',
 'SP50020dChange',
 'NASDAQCOM20dChange',
 'DCOILBRENTEU20dChange',
 'DHHNGSP20dChange',
 'DJFUELUSGULF60dChange',
 'DJFUELUSGULF20dChange',
 'T10YFF20dChange',
 'BAMLCC7A01015YTRIV60dChange',
 'BAMLCC7A01015YTRIV20dChange',
 'NASDAQCOM60dChange',
 'DBAA60dChange',
 'T10YFF60dChange',
 'DBAA20dChange',
 'DCOILWTICO60dChange',
 'DCOILWTICO20dChange',
 'T10Y2Y60dChange',
 'DGASUSGULF60dChange',
 'LBMA/GOLD20dChange',
 'LBMA/GOLD60dChange'
]
                                                                 

lgb_params = {
                    'objective': 'multiclass',
                    'metric': 'multi_logloss',
                    'num_classes': 5,
                } 

In [7]:
TOTAL_COST = 0.015 # %
num_periods_annually = 251 # количество торговых дней в году
risk_free_rate = 0.02/12 # безрисковая месячная ставка
TARGET = 'target'
BETA = 0
TICKERS_TO_TRADE = 40 ### сколько тикеров покупаем каждый месяц
years_to_train = 15 ### протестировано и выбрано
first_year = pd.DatetimeIndex([sorted(quotes.date.unique())[0]]).year[0]
last_year = first_year + years_to_train

CAPITAL = 1000000
START_CAPITAL = CAPITAL
CAPITAL_HISTORY = {np.datetime64(datetime.datetime(last_year,4,30)): CAPITAL} 

quotes['volume_USD'] = quotes['close']*quotes['volume'] # фильтр только на ликвидные акции 
VOLUME_TRESHOLD = 5000 # фильтр только на ликвидные акции c оборотом больше 5k долларов в посдедний день месяца

while last_year <= 2021-1:
  START_TRAIN = datetime.datetime(first_year,4,30) 
  END_TRAIN = datetime.datetime(last_year,4,30)
  END_TEST = datetime.datetime(last_year+1,4,30)
  print('TRAIN END:', last_year, 'TRAIN START:', last_year - years_to_train, 'TEST PERIOD:', last_year+1)
  first_year += 1
  last_year += 1
  ##########################################################
  train_mask = (quotes['date']>=START_TRAIN)&(quotes['date']<=END_TRAIN)&(-quotes['symbol'].isin(skip_symbols))
  test_mask = (quotes['date']>END_TRAIN)&(quotes['date']<=END_TEST)&(-quotes['symbol'].isin(skip_symbols))&(quotes['volume_USD']>VOLUME_TRESHOLD)

  train_data_lgb = lgb.Dataset(quotes[train_mask][features_columns], label=quotes[train_mask][TARGET])
  test_data_lgb = lgb.Dataset(quotes[test_mask][features_columns], label=quotes[test_mask][TARGET])

  estimator = lgb.train(lgb_params, 
                        train_data_lgb,
                        valid_sets = [train_data_lgb, test_data_lgb],
                        verbose_eval = 200)
  
  test_data = quotes[test_mask]
  test_dates = [date for date in list(test_data[test_mask]['date'].sort_values().unique())] # определяем список дат на которые нужно тест провести
  nasdaq = test_data[['date','NASDAQCOM']].drop_duplicates().sort_values(by=['date']).set_index('date').dropna() # benchmark
  nasdaq['nasdaq_pct_change'] = nasdaq['NASDAQCOM'].astype(float).pct_change().shift(-1) # benchmark
  nasdaq = nasdaq.reset_index() # benchmark

  for date in test_dates:
    # делаем срез для торговой даты
    test_data['volume_USD'] = test_data['close']*test_data['volume']
    sub_df = test_data[(test_data['date']==date)&(test_data['volume_USD']>VOLUME_TRESHOLD)] #
    sub_nasdaq = nasdaq[nasdaq['date']==date] # делаем срез по насдаку чтобы узнать изменение за следующий месяц по нему

    # делаем предсказание для торговой даты
    result = estimator.predict(sub_df[features_columns])
    # вывод выбранной категории в отдельный столбец
    final_result = []
    for line in result:
      final_result.append(list(line).index(max(line)))
    sub_df['prediction'] = final_result
    # и мэпим вероятности классов для каждого тикера в этот день
    sub_df.loc[:,'prob_0'] = [sublist[0] for sublist in result]
    sub_df.loc[:,'prob_1'] = [sublist[1] for sublist in result]
    sub_df.loc[:,'prob_2'] = [sublist[2] for sublist in result]
    sub_df.loc[:,'prob_3'] = [sublist[3] for sublist in result]
    sub_df.loc[:,'prob_4'] = [sublist[4] for sublist in result]

    # делаем столбец где отмечаем торговать или нет
    sub_df['is_trade_long'] = sub_df['prediction']==0
    # sub_df['is_trade_short'] = sub_df['prediction']==4

    mask_long = sub_df['is_trade_long']==True # отбираем все строки где стратегия помэпила лонг
    symbols_long = sub_df[mask_long][['symbol', 'is_trade_long', 'prob_0']].sort_values(by=['prob_0']).tail(TICKERS_TO_TRADE)['symbol'].values # отбираем 40 акций с наибольшей верояютностью к росту
    long_mask = sub_df['symbol'].isin(symbols_long)

    if len(symbols_long)>5:
      res_long = sub_df['price_change_next_calendar_month'][long_mask].mean()-TOTAL_COST
    else:
      # принимаем решение не торговать, а вкладываем в безрисковый процент
      res_long = risk_free_rate
    
    res = res_long

    CAPITAL = CAPITAL * (1+res_long)
    CAPITAL_HISTORY[date] = CAPITAL

# Сохраняем самую последнюю обученную модель
with open('{}lgbm_estimator.pickle'.format(DIRECTORY), 'wb') as f:
  pickle.dump(estimator, f)

# сделали прогон, теперь считаем метрики по получившемуся портфелю
portfolio_var = np.std(pd.DataFrame(CAPITAL_HISTORY.values()).pct_change())
df = pd.DataFrame(CAPITAL_HISTORY.values()).pct_change()
df = list(df[df.values<0][0])
negative_strategie_var = np.std(df)
portfolio_profit = (pd.DataFrame(CAPITAL_HISTORY.values()).pct_change()).mean()

STRATEGY_SHARP = round(float(portfolio_profit/portfolio_var)*math.sqrt(12),3) # Коэффициент шарпа годовой
STRATEGY_RESULT = (CAPITAL - START_CAPITAL) / START_CAPITAL # во сколько раз вырос депозит 
STRATEGY_VAR =  round(float(portfolio_var),3) # VAR депозита
STRATEGY_BETA = cal_beta(pd.DataFrame(list(CAPITAL_HISTORY.values()), index=CAPITAL_HISTORY.keys())) # считаем бетту с насдак композит

print('VAR', round(float(portfolio_var),3), 
      'PROFIT',  round(float(portfolio_profit),3),
      'SHARP', round(float(portfolio_profit/portfolio_var)*math.sqrt(12),3),
      'RESULT', round(STRATEGY_RESULT,2),
      'BETA', round(STRATEGY_BETA, 2))


# рисуем графики
df = pd.DataFrame(list(CAPITAL_HISTORY.values()), index=CAPITAL_HISTORY.keys())
window = 252
Roll_Max = df[0].rolling(window, min_periods=1).max()
Daily_Drawdown = df[0]/Roll_Max - 1.0
Max_Daily_Drawdown = Daily_Drawdown.rolling(window, min_periods=1).min()

fig, ax = plt.subplots(2, figsize=(10,7), sharex=True)
ax[0].plot(np.log(pd.DataFrame(list(CAPITAL_HISTORY.values()), index=CAPITAL_HISTORY.keys())),linewidth=1) 
fig.suptitle('Result and Drawdown, END TRAIN YEAR: {}, VAR {}, PROFIT {}, SHARP {}, BETA {}'.format(END_TRAIN, 
                                                                                            round(float(portfolio_var),3), 
                                                                                            round(float(portfolio_profit),3), 
                                                                                            round(float(portfolio_profit/portfolio_var)*math.sqrt(12),3),
                                                                                            round(STRATEGY_BETA, 2)))

ax[1].plot(Daily_Drawdown)
ax[1].plot(Max_Daily_Drawdown)

In [ ]:
# Для того, чтобы сделать предсказание на последнюю доступную дату, надо подготовить датасет. Здесь загружается датасет подготовленный для реал тайм предсказания
with open('{}final_df_last_year.pickle'.format(DIRECTORY), 'rb') as f: 
  quotes_final = pickle.load(f)

with open('{}lgbm_estimator.pickle'.format(DIRECTORY), 'rb') as f: 
  estimator = pickle.load(f)

In [ ]:
# делаем срез для торговой даты
DATE_TO_TRADE = '2021-05-28'
VOLUME_TRESHOLD = 5000 # фильтр только на ликвидные акции c оборотом больше 5k долларов в посдедний день месяца чтобы отсечь совсем неликвид
TICKERS_TO_TRADE = 40 # сколько акций набираем в портфель

sub_df = quotes_final[quotes_final['date']==DATE_TO_TRADE]

# делаем предсказание для торговой даты
result = estimator.predict(sub_df[features_columns])
# вывод выбранной категории в отдельный столбец
final_result = []
for line in result:
  final_result.append(list(line).index(max(line)))
sub_df['prediction'] = final_result
# и мэпим вероятности классов для каждого тикера в этот день
sub_df.loc[:,'prob_0'] = [sublist[0] for sublist in result]
sub_df.loc[:,'prob_1'] = [sublist[1] for sublist in result]
sub_df.loc[:,'prob_2'] = [sublist[2] for sublist in result]
sub_df.loc[:,'prob_3'] = [sublist[3] for sublist in result]
sub_df.loc[:,'prob_4'] = [sublist[4] for sublist in result]

sub_df['is_trade_long'] = sub_df['prediction']==0

mask_long = sub_df['is_trade_long']==True # отбираем все строки где стратегия помэпила лонг
mask_volume = sub_df['volume']*sub_df['close']>VOLUME_TRESHOLD # только ликвидные акции с оборотм больше 5к долларов в день
symbols_long = sub_df[mask_long&mask_volume][['symbol', 'is_trade_long', 'prob_0']].sort_values(by=['prob_0']).tail(TICKERS_TO_TRADE)['symbol'].values # отбираем 40 акций с наибольшей верояютностью к росту 
symbols_long # показать все отобранные тикеры

## Реал тайм тестирование стрегии, выбор тикеров на июнь

Ниже название тикера и цена открытия - клоуз 1 июня

TICKER: CLOSE AT 01/06/21

CXDC: 0.64

VMEO: 42.84

BCYPW: 0.67

HCICW: 1.2

BIMI: 1.32

ODT: 3.66

ACAD: 22.35

YJ: 1.82

FAMI: 0.44

EBON: 3.41

NRBO: 2.88

PAQCW: 0.69

SMID: 18.97

MGI: 9.7

RAACW: 1.47

DCRNW: 1.37

BCEL: 9.18

LYRA: 7.63

ALACW: 0.3181

IOVA: 18.27

GIFI: 4.92

SGTX: 11.71

PSTI: 3.82

ZVO: 2.31

LOTZ: 4.98

METX: 0.97

RELI: 3.98

DCPH: 33.56

NKLA: 15.44

EVFM: 0.897

PROG: 2.86

EVOK: 1.36

ICPT: 17.16

EZGO: 4.17

AEGN: 5.5

HYMCW: 0.63

SEII: 0.03

DUO: 3.3

QK: 0.992

NBRV: 1.39
